In [15]:
%pip install rdkit
%pip install pyvista

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 MB 7.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from io import BytesIO
import os

import numpy as np
import msms.wrapper as msms
import requests

from rdkit import Chem

# Check the installation

## Is msms available?
msms needs to be in the PATH. We can check this using `msms_available`. If msms does not exist (or if we delete the PATH), it will return False.

In [2]:
print("Can find msms?", msms.msms_available())

Can find msms? True


In [3]:
path = os.environ['PATH']
os.environ['PATH'] = ".."
print("With PATH set to empty:")
print("Can find msms?", msms.msms_available())
os.environ['PATH'] = path

With PATH set to empty:
Can find msms? True


## Get help

In [4]:
print(msms.help())

Usage : msms parameters 
  -probe_radius float : probe sphere radius, [1.5]
  -density float      : surface points density, [1.0]
  -hdensity float     : surface points high density, [3.0]
  -surface <tses,ases>: triangulated or Analytical SES, [tses]
  -no_area            : turns off the analytical surface area computation
  -socketName servicename : socket connection from a client
  -socketPort portNumber : socket connection from a client
  -xdr                : use xdr encoding over socket
  -sinetd             : inetd server connection
  -noh                : ignore atoms with radius 1.2
  -no_rest_on_pbr     : no restart if pb. during triangulation
  -no_rest            : no restart if pb. are encountered
  -if filename        : sphere input file
  -of filename        : output for triangulated surface
  -af filename        : area file
  -no_header         : do not add comment line to the output
  -free_vertices      : turns on computation for isolated RS vertices
  -all_components

# Get a structure
**We need:**
* coordinates (in this case, from the RCSB pdb)
* radii (in this case, using the mBondi2 definition as used in the Ambertools)

In [5]:
response = requests.get("https://files.rcsb.org/ligands/download/5P8_model.sdf")
lorlatinib = next(Chem.ForwardSDMolSupplier(BytesIO(response.content)))

In [6]:
points = lorlatinib.GetConformer(0).GetPositions()
points -= points.mean(0)

In [7]:
MBONDI2_RADII = {
    "C": 1.7,
    "N": 1.55,
    "O": 1.8,
    "Cl": 1.5,
    "Si": 2.1,
    "P": 1.85,
    "S": 1.8,
    "Br": 1.7,
}

def get_mbondi2_radii(mol):
    """Return the mBondi2 radii of a mol as a list"""
    periodic_table = Chem.GetPeriodicTable()
    out = []
    for i_atom, atom in enumerate(mol.GetAtoms()):
        elem = periodic_table.GetElementSymbol(atom.GetAtomicNum())
        if elem in MBONDI2_RADII:
            radius = MBONDI2_RADII[elem]
        elif elem == "H":
            bonded = atom.GetNeighbors()[0]
            bonded_elem = periodic_table.GetElementSymbol(bonded.GetAtomicNum())
            if bonded_elem == "N":
                radius = 1.3
            else:
                radius = 1.2
        else:
            radius = 1.5
        out.append(radius)
    return np.array(out)

In [8]:
radii = get_mbondi2_radii(lorlatinib)

# Run MSMS
## Usage
* Pairs of arguments can be added as `kwargs`, usually like `density=2.0` or `probe_radius=1.0`
* Further msms arguments can be added as `*args`, like `"-free_vertices"`
## Output format
* msms_out.vertices contains all information on vertices (position, normals, type etc.)
* Best split it into several numpy arrays

In [9]:
msms_out = msms.run_msms(points, radii, density=5.0, probe_radius=1)
verts = msms_out.get_vertex_positions()
normals = msms_out.get_vertex_normals()
faces = msms_out.get_face_indices()

# Visualize
(NOTE THIS SECTION WON'T WORK AS WRITTEN CURRENTLY VIA MyBinder, as spelled out [here 'Remote JupyterHubs'](https://tutorial.pyvista.org/tutorial/00_jupyter/index.html#remote-jupyterhubs) because a remote JupyterHub is involved. At this point, I don't know enough about what type of data this is to see how to connect it. But I think it should be possible given I know ipyvolume works in Jupyter and https://github.com/InsightSoftwareConsortium/itk-jupyter-widgets .I note that VTK should work in MyBinder as I'ves used it before, see even the launch badge [here](https://github.com/trungleduc/jupyterview).)

If `pyvista` (or a similar package) is installed, the surface can be visualized in a Jupyter Notebook

Note: this sometimes crashes when using `Run all cells`



In [10]:
import pyvista

In [11]:
def for_pyvista(arr):
    out = []
    for row in arr:
        out.append(len(row))
        out.extend(row)
    return out

In [12]:
surf = pyvista.PolyData(verts, faces=for_pyvista(faces))
surf.plot()

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

# Minimal usage example

Use `msms` to compute the surface area of a unit sphere.

* The SES and SAS are analyical.
* The volume is numerical. In the case of a single sphere, it is always too small, but converges with high density.

In [13]:
xyz = [[0., 0., 0.]]
radii = [1.]
print('expected SES', 4*np.pi)
print('expected SAS', 4*np.pi * 2.5**2) # 2.5 = radius + probe_radius
print('expected volume', 4/3*np.pi)
msms.run_msms(xyz, radii, density=2).extract_ses_sas_vol()

expected SES 12.566370614359172
expected SAS 78.53981633974483
expected volume 4.1887902047863905


SizeDescriptors(ses=12.566, sas=78.54, volume=3.082)